In [8]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import imageio
import os

In [10]:
# Open the NetCDF file
dataset_path = 'data/CERES_EBAF-TOA_Ed4.2_Subset_202008-202308.nc'
data = nc.Dataset(dataset_path)

# Print all variable names and their descriptions
for var in data.variables:
    print(var, data.variables[var].long_name)

# Close the NetCDF file
data.close()

lon Longitude
lat Latitude
time Time
toa_sw_all_mon Top of The Atmosphere Shortwave Flux, All-Sky conditions, Monthly Means
toa_lw_all_mon Top of The Atmosphere Longwave Flux, All-Sky conditions, Monthly Means
toa_net_all_mon Top of The Atmosphere Net Flux, All-Sky conditions, Monthly Means
toa_sw_clr_c_mon Top of The Atmosphere Shortwave Flux, Clear-Sky (for cloud-free areas of region) conditions, Monthly Means
toa_lw_clr_c_mon Top of The Atmosphere Longwave Flux, Clear-Sky (for cloud-free areas of region) conditions, Monthly Means
toa_net_clr_c_mon Top of The Atmosphere Net Flux, Clear-Sky (for cloud-free areas of region) conditions, Monthly Means
solar_mon Incoming Solar Flux, Monthly Means
cldarea_total_daynight_mon Cloud Area Fraction, Daytime-and-Nighttime conditions, Monthly Means
cldpress_total_daynight_mon Cloud Effective Pressure, Daytime-and-Nighttime conditions, Monthly Means
cldtemp_total_daynight_mon Cloud Effective Temperature, Daytime-and-Nighttime conditions, Monthly M

In [15]:
def plot_monthly_radiation(variable_name, time_index, radiation_data, lat, lon, output_folder):
    lon2d, lat2d = np.meshgrid(lon, lat)
    fig = plt.figure(figsize=(10, 7))
    ax = plt.axes(projection=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_global()

    # Determine color scale limits based on the variable being plotted
    if "net" in variable_name:
        vmin, vmax = -450, 450  # For net radiation, we use a range from -450 to 450
        cmap = 'magma'  # Use a diverging colormap for net radiation
    else:
        vmin, vmax = 0, 450  # For shortwave and longwave, we use a range from 0 to 450
        cmap = 'magma'  # Use a sequential colormap for shortwave and longwave

    # Create the color mesh with the determined scale and colormap
    radiation_plot = ax.pcolormesh(lon2d, lat2d, radiation_data[time_index, :, :], cmap=cmap, vmin=vmin, vmax=vmax, transform=ccrs.PlateCarree())
    cbar = plt.colorbar(radiation_plot, orientation='vertical', shrink=0.7)
    cbar.set_label('Radiation (W/m^2)')

    ax.gridlines(draw_labels=True)
    plt.title(f'{variable_name} {time_index//12 + 2020}-{time_index%12 + 1:02d}')

    # Create the output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save the figure
    plt.savefig(os.path.join(output_folder, f'{variable_name}_{time_index//12 + 2020}_{time_index%12 + 1:02d}.png'), dpi=300)
    
    # Close the plot to save memory
    plt.close(fig)

# Function to create an animation from plots
def create_animation(variable_name, output_folder, image_folder, frame_duration):
    images = []
    for file_name in sorted(os.listdir(image_folder)):
        if file_name.startswith(variable_name) and file_name.endswith('.png'):
            file_path = os.path.join(image_folder, file_name)
            images.append(imageio.imread(file_path))
    output_gif_path = os.path.join(output_folder, f'{variable_name}_Animation.gif')
    imageio.mimsave(output_gif_path, images, duration=frame_duration)
    print(f"Animated GIF for {variable_name} saved as {output_gif_path}")

# Main function to process the data and create animations
def process_and_create_animations(variable_names, dataset_path, output_folder, frame_duration=0.8):
    data = nc.Dataset(dataset_path)
    lat = data.variables['lat'][:]
    lon = data.variables['lon'][:]
    os.makedirs(output_folder, exist_ok=True)
    
    for variable_name in variable_names:
        radiation_variable = data.variables[variable_name][:]
        for time_index in range(radiation_variable.shape[0]):
            plot_monthly_radiation(variable_name, time_index, radiation_variable, lat, lon, output_folder)
        create_animation(variable_name, output_folder, output_folder, frame_duration)

    data.close()

# Define variables and dataset path
variable_names = ['toa_sw_all_mon', 'toa_lw_all_mon', 'toa_net_all_mon']
dataset_path = 'data/CERES_EBAF-TOA_Ed4.2_Subset_202008-202308.nc'
output_folder = 'CERES_plots'

# Call the main function
process_and_create_animations(variable_names, dataset_path, output_folder, frame_duration=0.5)

/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:52: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:52: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:39: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_sw_all_mon saved as CERES_plots/toa_sw_all_mon_Animation.gif


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:52: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:52: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:39: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_lw_all_mon saved as CERES_plots/toa_lw_all_mon_Animation.gif


/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:52: UserWarning: WARNING: valid_min not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:52: UserWarning: WARNING: valid_max not used since it
cannot be safely cast to variable data type
  radiation_variable = data.variables[variable_name][:]
/var/folders/nh/9y501_nj2x56h2t3ch9_05f40000gn/T/ipykernel_48755/3952322648.py:39: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(file_path))


Animated GIF for toa_net_all_mon saved as CERES_plots/toa_net_all_mon_Animation.gif
